*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
# import libraries
#try:
  # %tensorflow_version only exists in Colab.
  #!pip install tf-nightly
#except Exception:
  #pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
#!pip install tensorflow-datasets
#import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.7.0


In [2]:
# get data files
#!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
#!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:
train = pd.read_csv(train_file_path, sep = '\t', names = ['class', 'text'])
test = pd.read_csv(test_file_path, sep = '\t', names = ['class', 'text'])

In [4]:
labels = []
for i in range(train.shape[0]):
    if train.loc[i, 'class'] == 'ham':
        labels.append(0)
    else:
        labels.append(1)
train['labels'] = labels 
train

,class,text,labels
0,ham,ahhhh...just woken up!had a bad dream about u ...,0
1,ham,you can never do nothing,0
2,ham,"now u sound like manky scouse boy steve,like! ...",0
3,ham,mum say we wan to go then go... then she can s...,0
4,ham,never y lei... i v lazy... got wat? dat day ü ...,0
...,...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...,0
4175,ham,what do u reckon as need 2 arrange transport i...,0
4176,spam,free entry into our £250 weekly competition ju...,1
4177,spam,-pls stop bootydelious (32/f) is inviting you ...,1


In [5]:
labels2 = []
for i in range(test.shape[0]):
    if test.loc[i, 'class'] == 'ham':
        labels2.append(0)
    else:
        labels2.append(1)
test['labels'] = labels2
test

,class,text,labels
0,ham,i am in hospital da. . i will return home in e...,0
1,ham,"not much, just some textin'. how bout you?",0
2,ham,i probably won't eat at all today. i think i'm...,0
3,ham,don‘t give a flying monkeys wot they think and...,0
4,ham,who are you seeing?,0
...,...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...,0
1388,ham,"what will we do in the shower, baby?",0
1389,ham,where are you ? what are you doing ? are yuou ...,0
1390,spam,ur cash-balance is currently 500 pounds - to m...,1


In [6]:
def get_ds(dataframe, label, BATCH_SIZE = 10, test = False, BUFFER_SIZE = 100):
    ds = tf.data.Dataset.from_tensor_slices((dataframe, label))
    AUTOTUNE = tf.data.AUTOTUNE
    if test:
        ds =  ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
        return ds
    else:
        ds = ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
        return ds
text_train_ds = get_ds(train['text'], train['labels'])
text_test_ds = get_ds(test['text'], test['labels'], test = True)
#ds.labels = ('ham', 'spam')

In [7]:
max_f = 1000
seq_len = 1000
vectorize = keras.layers.TextVectorization(
            max_tokens = max_f,
            output_sequence_length=seq_len)
train_text = text_train_ds.map(lambda x, y:x)
vectorize.adapt(train_text)

def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize(text), label

In [8]:
embed_dim = 64
model = tf.keras.Sequential([
    keras.layers.Embedding(max_f + 1, embed_dim,mask_zero = True),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(64),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1)
])
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer= 'Adam', metrics = tf.metrics.BinaryAccuracy(threshold=0.0))

In [9]:
train_ds = text_train_ds.map(vectorize_text)
test_ds = text_test_ds.map(vectorize_text)

In [ ]:
hist = model.fit(train_ds,epochs=5, validation_data = test_ds)

In [40]:
text_model = tf.keras.Sequential([
    vectorize,
    model,
    keras.layers.Activation('sigmoid')
])
text_model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer= 'Adam', metrics = 'accuracy')

In [45]:
text_model.predict(["our new mobile video service is live. just install on your phone to start watching."])[0]

array([1.], dtype=float32)

In [48]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    text = [pred_text]
    #print(text)
    prediction = []
    prob = text_model.predict(text)[0]
    prediction.append(prob[0])
    if prob < 0.5:
        prediction.append('ham')
    elif prob > 0.5:
        prediction.append('spam')
    return (prediction)

pred_text = "our new mobile video service is live. just install on your phone to start watching."

prediction = predict_message(pred_text)
print(prediction)

[1.0, 'spam']


In [49]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
    test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[1] != ans:
              passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
